In [10]:
import random

def generateData(machines=4, jobs=7, seed=1899):
    PROC = [ [ 0 for m in range(machines) ] for j in range(jobs) ]
    random.seed(seed)
    for j in range(jobs):
        for m in range(machines):
            PROC[j][m] = random.randint(1,9)
    return PROC

In [11]:
PROC = generateData(1899)

In [13]:
PROC

[[3,
  1,
  4,
  5,
  2,
  4,
  9,
  3,
  2,
  8,
  8,
  4,
  2,
  5,
  3,
  3,
  8,
  1,
  7,
  4,
  2,
  3,
  3,
  8,
  5,
  8,
  3,
  7,
  7,
  5,
  1,
  8,
  8,
  1,
  6,
  3,
  1,
  2,
  8,
  6,
  6,
  4,
  8,
  8,
  1,
  7,
  2,
  2,
  5,
  7,
  9,
  7,
  7,
  6,
  8,
  1,
  8,
  9,
  6,
  5,
  8,
  7,
  3,
  4,
  3,
  1,
  8,
  2,
  6,
  9,
  2,
  5,
  6,
  7,
  4,
  3,
  1,
  8,
  9,
  3,
  4,
  4,
  4,
  4,
  9,
  1,
  2,
  5,
  9,
  4,
  2,
  7,
  8,
  1,
  4,
  3,
  5,
  4,
  2,
  1,
  2,
  5,
  3,
  5,
  8,
  6,
  7,
  6,
  9,
  4,
  9,
  6,
  7,
  2,
  8,
  2,
  7,
  6,
  6,
  2,
  2,
  4,
  5,
  8,
  3,
  2,
  7,
  3,
  5,
  5,
  6,
  4,
  8,
  2,
  4,
  7,
  4,
  8,
  3,
  3,
  3,
  4,
  2,
  1,
  4,
  6,
  3,
  8,
  9,
  5,
  4,
  2,
  5,
  9,
  1,
  5,
  6,
  1,
  9,
  8,
  4,
  6,
  9,
  7,
  8,
  7,
  6,
  3,
  9,
  1,
  2,
  5,
  9,
  9,
  7,
  4,
  1,
  5,
  1,
  9,
  2,
  9,
  4,
  6,
  4,
  2,
  3,
  8,
  7,
  4,
  8,
  3,
  5,
  3,
  3,
  4,
  8,
  8,
  5,
  1,


In [14]:
import pulp

def IP(PROC):

    steps = [ len(job) for job in PROC ]
    assert(min(steps)==max(steps))
    N_MACHINES = len(PROC[0])
    N_JOBS = len(PROC)


    def val(x):
        return int(pulp.value(x))

    def proc(job, machine):
        return pulp.lpSum( [ PROC[j][machine] * JJ[job][j] for j in range(N_JOBS) ] ) 

    JJ = pulp.LpVariable.dicts("J", (range(N_JOBS), range(N_JOBS)), lowBound=0, upBound=1, cat='Integer')
    Wait = pulp.LpVariable.dicts("WAIT", (range(N_JOBS), range(N_MACHINES)), lowBound=0, cat='Integer')
    Idle = pulp.LpVariable.dicts("IDLE", (range(N_JOBS), range(N_MACHINES)), lowBound=0, cat='Integer')
    Start = pulp.LpVariable.dicts("START", (range(N_JOBS), range(N_MACHINES)), lowBound=0, cat='Integer')
    Stop = pulp.LpVariable.dicts("STOP", (range(N_JOBS), range(N_MACHINES)), lowBound=0, cat='Integer')

    prob = pulp.LpProblem("JobScheduling",pulp.LpMinimize)
    prob += Stop[N_JOBS-1][N_MACHINES-1]

    # JJ is a permutation of the jobs
    for j in range(N_JOBS):
        prob += pulp.lpSum( [ JJ[j][jj] for jj in range(N_JOBS) ] ) == 1
        prob += pulp.lpSum( [ JJ[jj][j] for jj in range(N_JOBS) ] ) == 1

    for m in range(N_MACHINES):
        for j in range(N_JOBS):
            prob += pulp.lpSum( [ Idle[ji][m] + proc(ji, m) for ji in range(j) ] ) + Idle[j][m] == Start[j][m]

    for m in range(N_MACHINES):
        for j in range(N_JOBS):
            prob += pulp.lpSum( [ Wait[j][mi] + proc(j, mi) for mi in range(m) ] ) + Wait[j][m] == Start[j][m]

    for j in range(N_JOBS):
        for m in range(N_MACHINES):
            prob += Start[j][m] + proc(j,m) == Stop[j][m]

    solvers = pulp.listSolvers(onlyAvailable=True) 
    solver = pulp.getSolver(solvers[0], msg=0)
    prob.solve(solver)

    acc = []
    for j in range(N_JOBS):
        for jj in range(N_JOBS):
            if pulp.value(JJ[j][jj])==1:
                acc.append(jj)
                
    return acc, int(pulp.value(prob.objective))

In [15]:
import numpy as np

def totalTime(PROC, seq, log=False):
    
    def isPermutation(seq):
        for i in range(len(seq)):
            if i not in seq:
                return False
        return True
    
    steps = [ len(job) for job in PROC ]
    assert(len(PROC) == len(seq))
    assert(isPermutation(seq))
    assert(min(steps)==max(steps))
    n_machines = len(PROC[0])
    n_jobs = len(PROC)
    wait = np.zeros([n_jobs, n_machines], dtype=int)
    idle = np.zeros([n_jobs, n_machines], dtype=int)
    start = np.zeros([n_jobs, n_machines], dtype=int)
    stop  = np.zeros([n_jobs, n_machines], dtype=int)
    proc = np.zeros([n_jobs, n_machines], dtype=int)
    for job in range(n_jobs):
        proc[job] = PROC[seq[job]]
    for job in range(n_jobs):
        for machine in range(n_machines):
            start[job, machine] = max(stop[job-1, machine] if job>0 else 0, stop[job, machine-1] if machine>0 else 0)
            wait[job, machine] = start[job, machine] - (stop[job, machine-1] if machine>0 else 0)
            idle[job, machine] = start[job, machine] - (stop[job-1, machine] if job>0 else 0)
            stop[job, machine] = start[job, machine] + proc[job, machine]

    if log:

        row = '|---------|'
        for m in range(n_machines):
            row += f"------------|" \
                    f"------------|"
        print(row)
        
        row = '|         |'
        for m in range(n_machines):
            row += f"            |" \
                    f" Machine: {m:1d} |"
        print(row)
        
        for j in range(n_jobs):
            
            row = '|---------|'
            for m in range(n_machines):
                row += f"------------|" \
                       f"------------|"
            print(row)
            
            row = '|         |'
            for m in range(n_machines):
                row += f"       {' ':4s} | " \
                       f"Idle: {idle[j,m]:4d} |"
            print(row)
            
            row = '|---------|'
            for m in range(n_machines):
                row += f"------------|" \
                       f"------------|"
            print(row)
            
            row = '|         |'
            for m in range(n_machines):
                row += f"       {' ':4s} | " \
                       f"Start:{start[j,m]:4d} |"  
            print(row)
            
            row = f'| Job: {seq[j]:2d} |'
            for m in range(n_machines):
                row += f" Wait: {wait[j,m]:4d} | " \
                       f"Proc: {proc[j,m]:4d} |"
            print(row)
            
            row = '|         |'
            for m in range(n_machines):
                row += f"       {' ':4s} | " \
                       f"Stop: {stop[j,m]:4d} |"  
            print(row)
            
        row = '|---------|'
        for m in range(n_machines):
            row += f"------------|" \
                    f"------------|"
        print(row)

    
    return stop[n_jobs-1, n_machines-1]                                                       

In [16]:
PROC = generateData(machines=4, jobs=7, seed=1899)

Compute the optimal job schedule:

In [17]:
seq, proctime = IP(PROC)
print(seq)
print(proctime)

[5, 0, 1, 3, 6, 4, 2]
46


Check the processing time and print the schedule

In [18]:
totalTime(PROC, seq)

46

In [19]:
totalTime(PROC, seq, log=True)

|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Machine: 0 |            | Machine: 1 |            | Machine: 2 |            | Machine: 3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Idle:    0 |            | Idle:    2 |            | Idle:    5 |            | Idle:    8 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Start:   0 |            | Start:   2 |            | Start:   5 |            | Start:   8 |
| Job:  5 | Wait:    0 | Proc:    2 | Wait:    0 | Proc:    3 | Wait:    0 | Proc:    3 | Wait:    0 | Proc:    8 |
|         |            | Stop:    2 |            | Stop:    5 |            | Stop:    8 |            | Stop:   16 |
|---------|------------|------------|------------|------------|---------

46